# Analyze Data

Korte data analyse voor het verkenne van de yelp data voor het verslag van week 1.

In [64]:
# Imports
import pandas as pd
import os
import json
import random
from IPython.display import display
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
from sklearn.model_selection import train_test_split
import numpy as np
import operator

In [6]:
"""
This file loads the data from the data directory and shows you how.
Feel free to change the contents of this file!
Do ensure these functions remain functional:
    - get_business(city, business_id)
    - get_reviews(city, business_id=None, user_id=None, n=10)
    - get_user(username)
"""

import os
import json
import random

DATA_DIR = "../data"


def load_cities():
    """
    Finds all cities (all directory names) in ./data
    Returns a list of city names
    """
    return os.listdir(DATA_DIR)


def load(cities, data_filename):
    """
    Given a list of city names,
        for each city extract all data from ./data/<city>/<data_filename>.json
    Returns a dictionary of the form:
        {
            <city1>: [<entry1>, <entry2>, ...],
            <city2>: [<entry1>, <entry2>, ...],
            ...
        }
    """
    data = {}
    for city in cities:
        city_data = []
        with open(f"{DATA_DIR}/{city}/{data_filename}.json", "r") as f:
            for line in f:
                city_data.append(json.loads(line))
        data[city] = city_data
    return data


def get_business(city, business_id):
    """
    Given a city name and a business id, return that business's data.
    Returns a dictionary of the form:
        {
            name:str,
            business_id:str,
            stars:str,
            ...
        }
    """
    for business in BUSINESSES[city]:
        if business["business_id"] == business_id:
            return business
    raise IndexError(f"invalid business_id {business_id}")


def get_reviews(city, business_id=None, user_id=None, n=10):
    """
    Given a city name and optionally a business id and/or auser id,
    return n reviews for that business/user combo in that city.
    Returns a dictionary of the form:
        {
            text:str,
            stars:str,
            ...
        }
    """
    def should_keep(review):
        if business_id and review["business_id"] != business_id:
            return False
        if user_id and review["user_id"] != user_id:
            return False
        return True

    reviews = REVIEWS[city]
    reviews = [review for review in reviews if should_keep(review)]
    return random.sample(reviews, min(n, len(reviews)))


def get_user(username):
    """
    Get a user by its username
    Returns a dictionary of the form:
        {
            user_id:str,
            name:str,
            ...
        }
    """
    for city, users in USERS.items():
        for user in users:
            if user["name"] == username:
                return user
    raise IndexError(f"invalid username {username}")


CITIES = load_cities()
USERS = load(CITIES, "user")
BUSINESSES = load(CITIES, "business")
REVIEWS = load(CITIES, "review")
TIPS = load(CITIES, "tip")
CHECKINS = load(CITIES, "checkin")


In [7]:
''' Schoonmaken BUSINESSES'''
#business = citymerge(BUSINESSES)
#business = business[business['is_open'] == 1 & business['categories'].notna()]
#business['categories'] = business['categories'].str.split(', ')
#print(business.latitude.mean(), business.longitude.mean())

' Schoonmaken BUSINESSES'

In [8]:
# Functie om data van alle steden samen te voegen in 1 DataFrame
def citymerge(var):
    return pd.concat([pd.DataFrame(var[city]) for city in var]).reset_index()

In [31]:
CITY = 'westlake'
reviews = pd.read_json(DATA_DIR+'/'+CITY+'/review.json', lines=True)
businesses = pd.read_json(DATA_DIR+'/'+CITY+'/business.json', lines=True)
#businesses = citymerge(BUSINESSES)
reviews_train, reviews_test = train_test_split(reviews)#citymerge(REVIEWS)

In [32]:
# Creeer Utility Matrix en Mean Utility Matrix uit een variabele van data.py (REVIEWS, USERS, BUSINESSES, etc.)
def create_utility_matrix(df):

    utility_matrix  = pd.pivot_table(df, index='business_id', columns='user_id', values='stars')

    mean_ultility_matrix = utility_matrix - utility_matrix.mean()
    
    return utility_matrix, mean_ultility_matrix
    
utility_matrix, mean_utility_matrix = create_utility_matrix(reviews_train)

In [ ]:
def manhattan_similarity(matrix, id1, id2):
    """Compute manhattan similarity between two rows."""
    # compute distance
    distance = manhattan_distances(matrix, id1, id2)
    
    # if no distance could be computed (no shared features) return a similarity of 0
    if distance is np.nan:
        return 0
    
    # else return similarity
    return 1 / (1 + distance)

In [65]:
# Creeer Similarity Matrix uit Mean Utility Matrix
def similarity(mum):
    similarity_matrix_cosine = pd.DataFrame(cosine_similarity(mum.fillna(0)), index=mum.index, columns=mum.index).replace(0, np.nan)
    similarity_matrix_manhattan = 1 / (pd.DataFrame(manhattan_distances(mum.fillna(0)), index=mum.index, columns=mum.index).replace(0, np.nan) +1)
    similarity_matrix_euclidean = 1 / (pd.DataFrame(euclidean_distances(mum.fillna(0)), index=mum.index, columns=mum.index).replace(0, np.nan) +1)
    
    return similarity_matrix_cosine, similarity_matrix_manhattan, similarity_matrix_euclidean   
    
similarity_matrix_cosine, similarity_matrix_manhattan, similarity_matrix_euclidean = similarity(mean_utility_matrix)

In [34]:
def select_neighborhood(similarity_matrix, utility_matrix, target_user, target_business):
    """selects all items with similarity > 0"""
    # Controleer of target_user en target_business wel in de matrix zijn te vinden.
    if (target_business in similarity_matrix.index) and (target_user in utility_matrix.columns):

        # Maak een boolean mask van bedrijven die de gebruiker beoordeeld heeft met een similarity hoger dan 0.
        SelectedBusinesses = (similarity_matrix[target_business].index.isin(utility_matrix[target_user].dropna().index)) & (similarity_matrix[target_business] > 0)
    
        # return de bedrijven met de similarity door gebruik te maken van de eerder gecreeerde boolean mask.
        return similarity_matrix[target_business][SelectedBusinesses].sort_values(ascending = False)
    
    # Bij waarden die niet gevonden kunnen worden geef None terug.
    else:
        return pd.Series()

%time neighborhood = select_neighborhood(similarity_matrix, utility_matrix, 'e3fdrK1tMCwWLr76LFe-cA', 'ZTg8adZipR3QDoJmFZZqJw')
display(neighborhood)

Wall time: 7 ms


business_id
ZTg8adZipR3QDoJmFZZqJw    1.000000
eigkQ_PuRON8Se265NqQDQ    0.041096
oZH3Ee7Yjk7u8B4Ed0oVOg    0.040008
r0DureDzsHpzs_VZem5k7g    0.037465
irwDkp2eMP2x-4MfunRt8g    0.035356
EpJhRvkGDFE-GDPHM32klw    0.034690
PzuyoHj3-VrYK7N8ZestNA    0.032788
dVvij7VRh55dISu02I0IGw    0.028460
5MWWP4Kpmw0e8d2ib9G7Kg    0.027826
47me-6Zme7RYR0zEonfaHg    0.019205
0VJ8tBxOpD2OxuioVjaAxA    0.015957
spjaRNFn9Lmh4petKBuf5g    0.001452
Name: ZTg8adZipR3QDoJmFZZqJw, dtype: float64

In [57]:
def weighted_mean(neighborhood, utility_matrix, user_id):
    # Controleer of neighborhood wel een Series is en utility_matrix wel een DataFrame, anders return np.nan.
    if isinstance(neighborhood, pd.Series) and isinstance(utility_matrix, pd.DataFrame):
        # Als neighborhood of de utility_matrix leeg zijn return dan 0.
        if (neighborhood.empty) or (utility_matrix.empty):
            return np.nan
        
        # Controleer of user_id als kolom te vinden is, anders return 0.
        elif user_id in utility_matrix.columns:
    
            # Gebruik de bovenstaande formule om het gewogen gemiddelde voor de neighborhood te berekenen.
            return ((utility_matrix[user_id] * neighborhood).dropna().sum()) / (neighborhood.sum())

        else:
            return np.nan
    else:
        return np.nan
    
prediction = weighted_mean(neighborhood, utility_matrix, 'e3fdrK1tMCwWLr76LFe-cA')
print (prediction)

3.885708870994802


In [58]:
# Voorspel een score voor bedrijven en return deze als dict
def predictions(utility_matrix, similarity_matrix, user_id):
    predictdict = defaultdict()
    
    # Extract list of reviewed businesses to get new results
    review_list = utility_matrix[user_id][utility_matrix[user_id].notna()].index
    
    for business_id in similarity_matrix.index:
        if business_id in review_list:
            pass
        else:
            predictdict[business_id] = weighted_mean(select_neighborhood(similarity_matrix, utility_matrix, user_id, business_id), utility_matrix, user_id)
    
    return predictdict

In [83]:
# display(pd.read_json(DATA_DIR+'/'+CITY+'/user.json', lines=True).head())
# reviews[reviews['user_id'] == 'apP3CApEq6-z59tRLwEBYA']['business_id']

def predictions_combiner(ibcf_predictions, cb_predictions, ibcf_weight=0.8, reviews=reviews_train, businesses=businesses):
    hybrid_predictions = {}
    cb_weight = 1 - ibcf_weight
    for business_id in cb_predictions:
        if business_id in ibcf_predictions:
            ibcf_prediction = ibcf_predictions[business_id]
            cb_prediction = cb_predictions[business_id]
            hybrid_predictions[business_id] = ibcf_prediction*ibcf_weight + cb_prediction*cb_weight
            if ibcf_prediction == np.nan:
                hybrid_predictions[business_id] = cb_prediction
            if cb_prediction == np.nan:
                hybrid_predictions[business_id] = ibcf_prediction
        else:
            hybrid_predictions[business_id] = cb_prediction
            
    return  dict(sorted(hybrid_predictions.items(), key=operator.itemgetter(1), reverse=True))
    
def content_based_recommender(user_id, businesses=businesses):
    predicted_ratings = {}
    category_dict = make_category_dict(user_id)
    for business_id in businesses['business_id']:
        predicted_ratings[business_id] = content_based_predictor(user_id, business_id, category_dict)
    return predicted_ratings
    
def content_based_predictor(user_id, business_id, category_dict, reviews=reviews_train, businesses=businesses):
    ratings = []
    categories = businesses[businesses['business_id'] == business_id]['categories'].str.split(', ')
    for x in categories:
        for category in x:
            if category in category_dict:
                ratings.append(category_dict[category])
    if len(ratings) == 0:
        return np.nan
    else:
        return sum(ratings)/len(ratings)
        
def make_category_dict(user_id, reviews=reviews, businesses=businesses):
    category_dict = {}
    user_reviews = reviews[reviews['user_id'] == user_id]
    business_ids = user_reviews['business_id']
    for business_id in business_ids:
        categories = businesses[businesses['business_id'] == business_id]['categories'].str.split(', ')
        stars = user_reviews[user_reviews['business_id'] == business_id]['stars'].values[0]
        for x in categories:
            for category in x:
                if category in category_dict:
                    category_dict[category].append(stars)
                else:
                    category_dict[category] = [stars]
    for category in category_dict:
        category_dict[category] = sum(category_dict[category])/len(category_dict[category])
    return category_dict

%time cb_predictions = content_based_recommender('e3fdrK1tMCwWLr76LFe-cA')
%time ibcf_predictions = predictions(utility_matrix, similarity_matrix, 'e3fdrK1tMCwWLr76LFe-cA')
# cb_predictions
# ibcf_predictions
# only_icbf = predictions_combiner(ibcf_predictions, cb_predictions)
all_predictions = predictions_combiner(ibcf_predictions, cb_predictions)

Wall time: 842 ms
Wall time: 2.55 s


In [62]:
def select_items(predictions):
    pool = []
    safety = 0
    average = sum(list(predictions.values())[0:10])/10
    average -= 0.1 * average
    for prediction in predictions:
        if predictions[prediction] > average:
            pool.append(prediction)
        else:
            break
    if len(pool) < 10:
        pool = list(predictions.keys())[0:10]
    return random.sample(pool, k=10)

select_items(all_predictions)

['zq4i9ZtLebbvyWjKXec3NQ',
 'i7lFu1-iadoXW5Hn-JWaeg',
 'VZKWW2zQbk-rxwpAcque8w',
 'OD88wvH-9LxM_Gz4oHOIDg',
 '9vsqbJgjUqQNJpHrSj6jKw',
 'pIBX_IBixagX-2AzFPl77g',
 'LhW7pWkVgu_bAMiRhO3Wow',
 'UT0FUBRmpnGHE1U4Jpegxw',
 '-MsRvdPnuw6QuLn5Vxjruw',
 'yBPeUobSSaJQKaYYgiOKYA']

In [90]:
pd.Series(all_predictions).dropna()

i7lFu1-iadoXW5Hn-JWaeg    4.000000
nYvBZYg9rfqWFTYuxSVMdw    3.716189
2lcK3d4K7FU6O8wXdWzOmA    3.418317
mW28NmePxX7pZv8lCv7v2Q    3.384202
0ZN2MfHyjNIkCx7qJvVhDg    3.359787
KR2kRmHnRCaNzOUEGoB25w    2.766890
LhW7pWkVgu_bAMiRhO3Wow    3.900000
K5iqS0JXnKFFujZHIWQsag    3.233669
UT0FUBRmpnGHE1U4Jpegxw    3.900000
_v7lMUtdd6WlgKUEBLA_VA    3.043494
QcW0360vpeEilCLBlKEiLw    3.205307
IYZ5TSguf72Pq-ZJzZu0WQ    3.100000
bzdb1jJ1j8Qn_RVHY97FnA    3.001130
7dlCzYnXDaubTAfvgAX6sQ    3.446210
z58nyUVyDV-vC7nXFfvR5g    2.524324
op0sZT-TNyeTMw0m3HUUDQ    4.000000
5kCRty4p7tBwM9P7MAXgvA    3.591778
kkNWzhSpAjxm0zIxol3IzQ    3.848649
95Efv0xKoUsP5lvTnaeK5w    3.738281
x5lN8HdgDVWXwoL9N7MDoQ    3.564828
hTdV1_Xn1j6yhpIR7CgCJQ    4.282431
t10X85k_m8RuAZZgoMzvXg    4.000000
vkD7xW81EgCSWwiTvxtVAw    3.871036
EpJhRvkGDFE-GDPHM32klw    3.781081
SPb3oLIVZynmiS670Vo1kQ    3.547710
m9Cme6bUPuZFtqhN1AgmdQ    3.519102
b--I9Ed6N5P9fwEb2qaiqg    3.170135
BsLxCm_DMxCNVEPHoaj8hw    3.131985
yG9SpxEQVIhk6e1o3jbS

In [66]:
def predict_ratings_item_based(similarity, utility, test_data):
    # For loop door alle index waarden
    for ID in test_data.index:
        # Bereken neighbourhood
        neighborhood = select_neighborhood(similarity, utility, test_data.loc[ID, 'user_id'], test_data.loc[ID, 'business_id'])
        # Voeg het gewogen gemiddelde toe als predicted rating aan de test_data Dataframe
        test_data.loc[ID, 'predicted rating'] = weighted_mean(neighborhood, utility, test_data.loc[ID, 'user_id'])
    
    return test_data
    
predicted_item_based_cosine = predict_ratings_item_based(similarity_matrix_cosine, utility_matrix, review_test[['user_id', 'business_id', 'stars']])
predicted_item_based_manhattan = predict_ratings_item_based(similarity_matrix_manhattan, utility_matrix, review_test[['user_id', 'business_id', 'stars']])
predicted_item_based_euclidean = predict_ratings_item_based(similarity_matrix_euclidean, utility_matrix, review_test[['user_id', 'business_id', 'stars']])

def mse(predicted_ratings):
    # Bereken mse uit de formule hierboven
    return ((predicted_ratings['stars'] - predicted_ratings['predicted rating']).pow(2).sum()) / (predicted_ratings.shape[0])
    
print('Cosine Similarity MSE: '+str(mse(predicted_item_based_cosine)))
print('Manhattan Similarity MSE: '+str(mse(predicted_item_based_manhattan)))
print('Cosine Similarity MSE: '+str(mse(predicted_item_based_euclidean)))

c:\python\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\python\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Cosine Similarity MSE: 0.06741598823202663
Manhattan Similarity MSE: 0.6377814623080322
Cosine Similarity MSE: 0.6331693729793064


In [74]:
test = review_test[['user_id', 'business_id', 'stars']]
test[test['user_id'] == 'e3fdrK1tMCwWLr76LFe-cA']

,user_id,business_id,stars
7076,e3fdrK1tMCwWLr76LFe-cA,cbjF6szaq2orE0BplGAKEA,3.0
14487,e3fdrK1tMCwWLr76LFe-cA,5rb2tq_AhxQuRYu_bh464g,5.0
13902,e3fdrK1tMCwWLr76LFe-cA,irwDkp2eMP2x-4MfunRt8g,4.0
11987,e3fdrK1tMCwWLr76LFe-cA,r0DureDzsHpzs_VZem5k7g,3.0
9027,e3fdrK1tMCwWLr76LFe-cA,EpJhRvkGDFE-GDPHM32klw,3.0
12002,e3fdrK1tMCwWLr76LFe-cA,r0DureDzsHpzs_VZem5k7g,4.0


In [82]:
def predict_ratings_content_based(test_data):
    # For loop door alle index waarden
    for ID in test_data.index:
        # Bereken neighbourhood
        category_dict = make_category_dict(test_data.loc[ID, 'user_id'])
        test_data.loc[ID, 'predicted rating'] = content_based_predictor(test_data.loc[ID, 'user_id'], test_data.loc[ID, 'business_id'], category_dict)
    
    return test_data

predicted_content_based = predict_ratings_content_based(review_test[['user_id', 'business_id', 'stars']])

print (mse(predicted_content_based))

0.07891086730088631
